![Verne Academy Summit 2024](https://raw.githubusercontent.com/javendia/verne-academy-summit-2024/main/header.png)

## Paso 1: Ingesta de datos
- Leemos el fichero **Colors.csv**, especificando el formato (CSV) e indicando que el archivo contiene las cabeceras de las columnas
- Creamos la tabla delta destino en caso de no existir

In [10]:
from pyspark.sql.types import *
from delta.tables import *
from pyspark.sql.functions import when, lit, col

df = spark.read.load('Files/wwi/Colors.csv',
    format='csv',
    header=True
)

DeltaTable.createIfNotExists(spark) \
     .tableName("Colors") \
     .addColumn("ColorID", IntegerType()) \
     .addColumn("ColorName", StringType()) \
     .addColumn("LastUpdated", TimestampType()) \
     .execute()

deltaTable = DeltaTable.forPath(spark, 'Tables/colors')

StatementMeta(, dce7896c-18de-4f6a-a8d4-7e38689c819e, 12, Finished, Available)

## Paso 2: Transformación
- Seleccionamos las columnas necesarias
- Devolvemos un valor controlado para la columna **ColorName** en caso de ser nula o vacía

In [11]:
df = df.select("ColorID", "ColorName") \
        .withColumn("ColorName", when((col("ColorName").isNull() | (col("ColorName")=="")),lit("N/A")).otherwise(col("ColorName")))

StatementMeta(, dce7896c-18de-4f6a-a8d4-7e38689c819e, 13, Finished, Available)

## Paso 3: Creación de vista temporal
- Creamos la vista temporal **vw_colors** donde almacenar los datos tratados anteriormente

In [12]:
df.createOrReplaceTempView("vw_colors")

StatementMeta(, dce7896c-18de-4f6a-a8d4-7e38689c819e, 14, Finished, Available)

## Paso 4: Instrucción MERGE

- Si el valor del registro para la columna **ColorID** existe **(MATCHED)** en la tabla destino y alguna columna difiere del registro existente, actualizamos la fila
- Si el valor del registro para la columna **ColorID** no existe **(NOT MATCHED)** en la tabla destino, insertamos una nueva fila
- En caso de que el registro de la tabla destino no exista en el fichero origen, eliminamos esa fila

In [13]:
%%sql

MERGE INTO colors AS target
USING vw_colors AS source
ON target.ColorID = source.ColorID
WHEN MATCHED AND 
(
    target.ColorName <> source.ColorName
)
THEN
UPDATE SET
    target.ColorName = source.ColorName
    ,target.LastUpdated = CURRENT_TIMESTAMP()
WHEN NOT MATCHED THEN
INSERT (ColorID, ColorName, LastUpdated)
VALUES (source.ColorID, source.ColorName, CURRENT_TIMESTAMP())
WHEN NOT MATCHED BY SOURCE THEN
DELETE;

StatementMeta(, dce7896c-18de-4f6a-a8d4-7e38689c819e, 15, Finished, Available)

<Spark SQL result set with 1 rows and 4 fields>

## Paso 5: Eliminación vista temporal

In [14]:
spark.catalog.dropTempView("vw_colors")

StatementMeta(, dce7896c-18de-4f6a-a8d4-7e38689c819e, 16, Finished, Available)

True